Example / Test Main

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
import numpy as np

import os
import cProfile
from tqdm import tqdm

In [4]:
import toml


conf = toml.load("gde/config.toml")

In [5]:
conf

{'whoscored': {'leagues': ['GER-Bundesliga2',
   'GER-Bundesliga',
   'ENG-Premier League'],
  'seasons': [16, 17, 18, 19, 20, 21, 22]},
 'training': {'version': 0.09}}

In [6]:
ws = sd.WhoScored(
    #  
    leagues=["GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League"],
    seasons=[16, 17, 18, 19, 20, 21],
    # leagues=["GER-Bundesliga"],
    # seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=False,
)

[05/05/24 10:25:27] INFO     patching driver executable                                              ]8;id=885443;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/undetected_chromedriver/patcher.py\patcher.py]8;;\:]8;id=19850;file:///home/morten/.conda/envs/packing-report/lib/python3.10/site-packages/undetected_chromedriver/patcher.py#346\346]8;;\
                             /home/morten/.local/share/undetected_chromedriver/undetected_chromedriv               
                             er                                                                                    

In [7]:
schedule = ws.read_schedule().reset_index()
test_game = schedule["game_id"].values[0]
test_date = schedule[schedule["game_id"] == test_game]["date"].values[0]

In [8]:
schedule = schedule.sort_values("date")

In [9]:
schedule

,league,season,game,date,home_team,away_team,game_id,url,stage
4116,GER-Bundesliga2,1617,2016-08-05 Kaiserslautern-Hannover,2016-08-05 19:30:00,Kaiserslautern,Hannover,1090815,https://www.whoscored.com/Matches/1090815/Live...,NaN
4117,GER-Bundesliga2,1617,2016-08-06 Bochum-Union Berlin,2016-08-06 12:00:00,Bochum,Union Berlin,1090817,https://www.whoscored.com/Matches/1090817/Live...,NaN
4119,GER-Bundesliga2,1617,2016-08-06 Sandhausen-Fortuna Duesseldorf,2016-08-06 14:30:00,Sandhausen,Fortuna Duesseldorf,1090822,https://www.whoscored.com/Matches/1090822/Live...,NaN
4118,GER-Bundesliga2,1617,2016-08-06 Dynamo Dresden-Nuernberg,2016-08-06 14:30:00,Dynamo Dresden,Nuernberg,1090823,https://www.whoscored.com/Matches/1090823/Live...,NaN
4123,GER-Bundesliga2,1617,2016-08-07 Greuther Fuerth-1860 Munich,2016-08-07 12:30:00,Greuther Fuerth,1860 Munich,1090819,https://www.whoscored.com/Matches/1090819/Live...,NaN
...,...,...,...,...,...,...,...,...,...
2272,ENG-Premier League,2122,2022-05-22 Brighton-West Ham,2022-05-22 16:00:00,Brighton,West Ham,1549911,https://www.whoscored.com/Matches/1549911/Live...,NaN
2275,ENG-Premier League,2122,2022-05-22 Crystal Palace-Man Utd,2022-05-22 16:00:00,Crystal Palace,Man Utd,1549914,https://www.whoscored.com/Matches/1549914/Live...,NaN
2270,ENG-Premier League,2122,2022-05-22 Arsenal-Everton,2022-05-22 16:00:00,Arsenal,Everton,1549909,https://www.whoscored.com/Matches/1549909/Live...,NaN
2273,ENG-Premier League,2122,2022-05-22 Burnley-Newcastle,2022-05-22 16:00:00,Burnley,Newcastle,1549912,https://www.whoscored.com/Matches/1549912/Live...,NaN


In [10]:
from gde_utils.date_utils import to_datetime
from gde.database_io.db_handler import DB_handler
from game.game_timeline import GameTimeline

In [12]:
import logging
logger = logging.getLogger()
logger.disabled = True
dbh = DB_handler("/home/morten/Develop/packing-report/gde/GDE.db")
schedule = ws.read_schedule().reset_index()
schedule = schedule.sort_values("date")
for league, game, date, home in zip(schedule["league"].values, schedule["game_id"].values, schedule["date"].values, schedule["home_team"].values):
    date = to_datetime(date)
    game_timeline = GameTimeline(ws, game, date, league, dbh, 0.2, home)
    game_timeline.handle()

___